In [1]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math

In [2]:
train_all_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_all_df.reset_index(inplace=True)
train_all_df.head()

OSError: File b'train.csv' does not exist

In [ ]:
def featurize(df):
    df = df.copy()
    def featurizeStateHoliday(x):
        if x == 'a':
            return 1
        elif x == 'b':
            return 2
        elif x == 'c':
            return 3
        else:
            return x
    df['StateHoliday'] = df['StateHoliday'].map(lambda x: featurizeStateHoliday(x))
    

    def featurizeDate(ts): 
        features = {}
        features['cal:dayOfMonth'] = ts.day
        features['cal:dayOfWeek'] = ts.dayofweek
        features['cal:dayofyear'] = ts.dayofyear
        features['cal:month'] = ts.month
        features['cal:year'] = ts.year
        features['cal:weekofyear'] =  ts.weekofyear
        features['cal:quarter'] = ts.quarter

        return features

    date_features_series = df['Date'].map(lambda x: featurizeDate(x))
    date_features = DataFrame.from_records(date_features_series.tolist(), index=df['Date'])

    df = pandas.concat([df, date_features.reset_index()], axis=1)

    del df['Date']
    
    return df

In [ ]:
train_all_df = featurize(train_all_df)

train_df, test_df = cross_validation.train_test_split(train_all_df, train_size=.90)

train_individual_df, train_ensemble_df = cross_validation.train_test_split(train_df, train_size=.75)

train_all_df.head()

In [ ]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

# One Big Model

In [3]:
features = [col for col in train_individual_df.columns if col != 'Sales' and col != 'Customers']
regressor_big = ExtraTreesRegressor(n_estimators=100, n_jobs=-1)
regressor_big.fit(train_individual_df[features], train_individual_df['Sales'])
score(predicted=regressor_big.predict(test_df[features]), actual=test_df['Sales'])

NameError: name 'train_individual_df' is not defined

# Per store model


In [4]:

regressors = {}
features = [col for col in train_individual_df.columns if col != 'Sales' and col != 'Customers']
i = 0
for store in train_individual_df['Store'].unique():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_train_df = train_individual_df[train_individual_df['Store'] == store]
    
    regressor = ExtraTreesRegressor(n_estimators=50, n_jobs=-1)
    regressor.fit(store_train_df[features], store_train_df['Sales'])
    regressors[store] = regressor


NameError: name 'train_individual_df' is not defined

In [11]:
store_predictions = {}
for store in test_df['Store'].unique():
    
    features = [col for col in test_df.columns if col != 'Sales' and col != 'Customers']
    store_test_df = test_df[test_df['Store'] == store]
    store_predictions[store] = DataFrame.from_dict({"Predicted": regressors[store].predict(store_test_df[features]), "Actual": store_test_df['Sales']})

In [12]:
results = pandas.concat(store_predictions.values())

In [13]:
score(predicted=results.Predicted, actual=results.Actual)

0.15765781175452115

# Means

In [25]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means


train_individual_df = pandas.merge(train_individual_df, calculate_means(train_individual_df), on = ['Store','DayOfWeek','Promo'], how='left')

train_individual_df['Predict-mean'] = train_individual_df['Sales_y']
del train_individual_df['Sales_y']
train_individual_df['Sales'] = train_individual_df['Sales_x']
del train_individual_df['Sales_x']

train_individual_df.fillna(train_individual_df['Predict-mean'].mean(), inplace=True)



In [27]:

score(train_individual_df['Predict-mean'], actual=train_individual_df['Sales'])

0.20415715181130695

# Ensemble



In [14]:
train_ensemble_df_bk = train_ensemble_df.copy()

In [15]:
# Create ensemble training set
partials = []

features = [col for col in train_ensemble_df.columns if col != 'Sales' and col != 'Customers']
for store in train_ensemble_df['Store'].unique():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_train_df = train_individual_df[train_individual_df['Store'] == store]
    store_train_df['Predict-all_in_one'] = regressor_big.predict(store_train_df[features])
    store_train_df['Predict-by_store'] = regressors[store].predict(store_train_df[features])
    
    partials.append(store_train_df)
    

/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [16]:
ensemble_df = pandas.concat(partials)

In [17]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means


ensemble_df = pandas.merge(ensemble_df, calculate_means(ensemble_df), on = ['Store','DayOfWeek','Promo'], how='left')

ensemble_df['Predict-mean'] = ensemble_df['Sales_y']
del ensemble_df['Sales_y']
ensemble_df['Sales'] = ensemble_df['Sales_x']
del ensemble_df['Sales_x']

ensemble_df.fillna(ensemble_df['Predict-mean'].mean(), inplace=True)


In [18]:
partials = []

features = [col for col in test_df.columns if col != 'Sales' and col != 'Customers']
for store in test_df['Store'].unique():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_test_df = test_df[test_df['Store'] == store]
    store_test_df['Predict-all_in_one'] = regressor_big.predict(store_test_df[features])
    store_test_df['Predict-by_store'] = regressors[store].predict(store_test_df[features])
    
    partials.append(store_test_df)
test_ensemble_df = pandas.concat(partials)

/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300


In [19]:
features = [col for col in ensemble_df.columns if col != 'Sales' and col != 'Customers']
regressor_ensemble = LinearRegression(normalize="L1")
regressor_ensemble.fit(ensemble_df[features], ensemble_df['Sales'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize='L1')

In [20]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means


test_ensemble_df = pandas.merge(test_ensemble_df, calculate_means(ensemble_df), on = ['Store','DayOfWeek','Promo'], how='left')

test_ensemble_df['Predict-mean'] = test_ensemble_df['Sales_y']
del test_ensemble_df['Sales_y']
test_ensemble_df['Sales'] = test_ensemble_df['Sales_x']
del test_ensemble_df['Sales_x']

test_ensemble_df.fillna(test_ensemble_df['Predict-mean'].mean(), inplace=True)


In [21]:
# Fix order
test_ensemble_df = test_ensemble_df[ensemble_df.columns]

In [22]:
features = [col for col in test_ensemble_df.columns if col != 'Sales' and col != 'Customers']
score(predicted=regressor_ensemble.predict(test_ensemble_df[features]), actual=test_ensemble_df['Sales'])

1.9137178798810788

# Submit

In [ ]:
eval_df = DataFrame.from_csv("test.csv", parse_dates=['Date'])

In [ ]:
eval_df.reset_index(inplace=True)
eval_df.head()

In [ ]:
eval_df = featurize(eval_df)


In [ ]:
eval_df.columns

In [ ]:
eval_df.fillna(0, inplace=True)

In [ ]:
partials = []

features = [col for col in eval_df.columns if col != 'Sales' and col != 'Customers' and col != "Id"]
for store in eval_df['Store'].unique():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_test_df = eval_df[eval_df['Store'] == store]
    store_test_df['Predict-all_in_one'] = regressor_big.predict(store_test_df[features])
    store_test_df['Predict-by_store'] = regressors[store].predict(store_test_df[features])
    
    partials.append(store_test_df)
eval_df = pandas.concat(partials)

In [ ]:
del eval_df['Sales_x']
del eval_df['Sales_y']

In [ ]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means


eval_df = pandas.merge(eval_df, calculate_means(train_all_df), on = ['Store','DayOfWeek','Promo'], how='left')

eval_df['Predict-mean'] = eval_df['Sales']
del eval_df['Sales']

eval_df.fillna(eval_df['Predict-mean'].mean(), inplace=True)


In [ ]:
features = [col for col in test_ensemble_df.columns if col != 'Sales' and col != 'Customers']
predictions = regressor_ensemble.predict(eval_df[features])

In [ ]:
eval_df['Predictions'] = predictions

In [ ]:
ensemble_results = eval_df[['Id', 'Predictions']]
ensemble_results.columns=['Id', 'Sales']
ensemble_results.to_csv( "results-ensemble.csv", index = False )